In [1]:
import cv2
import numpy as np
import os
import random
import torch
import config

from model_structure import UNet
from tqdm import tqdm

In [2]:
# Load the PyTorch model
model = UNet()
model.load_state_dict(torch.load("./model_for_vasc.pth", map_location="cpu"))

# Set the model to evaluation mode
model.eval()

# Directory paths for input and output data
input_dir = "./preparations/data/indata"
output_dir = "./preparations/data/outdata"

# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

In [31]:
while True:
    # Choose a random pair of input and output files
    random_image_file = random.choice(image_files)

    # Construct the full file paths
    input_image_path = os.path.join(input_dir, random_image_file)
    output_label_path = os.path.join(output_dir, random_image_file)
    
    label_image = cv2.imread(output_label_path, cv2.IMREAD_GRAYSCALE)
    shape = label_image.shape
    random_x, random_y = random.randint(0, shape[0] - config.WIDTH - 1), random.randint(0, shape[1] - config.HEIGHT - 1)
    label_image = label_image[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
    if cv2.countNonZero(label_image) == 0:
        # continue
        pass
    img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
    mask = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
    input_image = img * (mask // 255)

    shape = input_image.shape
    break

# Perform inference using the model
with torch.no_grad():
    input_tensor = torch.from_numpy(input_image / 255).unsqueeze(0).float()
    output = model(input_tensor)
# Convert the output tensor to a NumPy array
output_np = output.numpy()
# Convert the output prediction to binary format and multiply by 255
print((np.unique(output_np).size))
binary_output = (output_np >= 0.5).astype(np.uint8)[0] * 255
# Display the input image, label, and binary prediction using OpenCV
cv2.imshow("Input Image", cv2.resize(input_image, (512, 512)))
cv2.imshow("Label Image", cv2.resize(label_image, (512, 512)))
print(cv2.resize(binary_output, (512, 512)).shape)
cv2.imshow("Model Prediction (Binary)", cv2.resize(binary_output, (512, 512)))
cv2.imshow("Correctness Image", cv2.resize((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255, (512, 512)))

TP = np.sum(np.multiply(binary_output, label_image))
TN = np.sum(np.multiply((255 - binary_output), (255 - label_image)))
FP = np.sum(np.multiply(binary_output, (255 - label_image)))
FN = np.sum(np.multiply((255 - binary_output), label_image))
print("TP:", TP)
print("TN:", TN)
print("FP:", FP)
print("FN:", FN)

print(TP / (TP + FN))
print(TN / (TN + FP))
print((TP + TN) / (TP + TN + FP + FN))

print("")
print("Positive Recall:", TP / (TP + FP))
print("Negative Recall:", TN / (TN + FN))

# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

4093
(512, 512)
TP: 0
TN: 4068
FP: 28
FN: 0
nan
0.9931640625
0.9931640625

Positive Recall: 0.0
Negative Recall: 1.0


C:\Users\atashitk\AppData\Local\Temp\ipykernel_10796\552638494.py:48: RuntimeWarning: invalid value encountered in scalar divide
  print(TP / (TP + FN))


In [8]:
def get_prediction(img, margin):
    if margin != 0:
        raise ValueError
    with torch.no_grad():
        input_tensor = torch.from_numpy(img / 255).unsqueeze(0).float()
        output = model(input_tensor)
    output_np = output.numpy()
    return output_np

In [20]:
input_dir = "./preparations/data/indata"
output_dir = "./preparations/data/outdata"

margin = 0
step = 16
# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

show_size = (512, 512)

random_image_file = random.choice(image_files)
print("Random image name:", random_image_file)
input_image_path = os.path.join(input_dir, random_image_file)
output_label_path = os.path.join(output_dir, random_image_file)

input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
label_image = cv2.imread(output_label_path, cv2.IMREAD_GRAYSCALE)

shape = label_image.shape

img = input_image[shape[0] % step:, shape[1]%step:]
mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

input_image = img * (mask // 255)
label_image = label_image[shape[0] % step:, shape[1]%step:]

shape = label_image.shape
count_times = np.zeros(shape)
total_times = np.zeros(shape)

arr1 = []
arr2 = []
for i in tqdm(range(0, shape[0] - config.HEIGHT, step), "Processing"):
    for j in range(0, shape[1] - config.WIDTH, step):
        
        slice_tmp = input_image[i : i + config.HEIGHT, j : j + config.WIDTH]
        # print(i, j, slice.shape)
        array_tmp = get_prediction(slice_tmp, margin)[0]
        # print(array_tmp.shape)
        count_times[i : (i + config.HEIGHT), j : (j + config.WIDTH)] += (array_tmp > 0.5)
        
        total_times[i : (i + config.HEIGHT), j : (j + config.WIDTH)] += 1

        

unique = np.unique(count_times)
confidence = (count_times + 0.0001) / (total_times + 0.0001)
binary_output = (np.floor(confidence * 255))

cv2.imshow("Input Image", cv2.resize(input_image, show_size))
cv2.imshow("Label Image", cv2.resize(label_image, show_size))
cv2.imshow("Model Prediction (Binary)", cv2.resize(binary_output, show_size))
cv2.imshow("Correctness Image", cv2.resize((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255, show_size))

TP = np.sum(np.multiply(binary_output, label_image))
TN = np.sum(np.multiply((255 - binary_output), (255 - label_image)))
FP = np.sum(np.multiply(binary_output, (255 - label_image)))
FN = np.sum(np.multiply((255 - binary_output), label_image))
print("TP:", np.round(TP / (TP + TN + FP + FN) * 100, 2), "%")
print("TN:", np.round(TN / (TP + TN + FP + FN) * 100, 2), "%")
print("FP:", np.round(FP / (TP + TN + FP + FN) * 100, 2), "%")
print("FN:", np.round(FN / (TP + TN + FP + FN) * 100, 2), "%")

print(TP / cv2.countNonZero(label_image) / 255 / 255)
print(TN / (shape[0] * shape[1] - cv2.countNonZero(label_image)) / 255 / 255)
print((TP + TN) / shape[0] / shape[1] / 255 / 255)

print("")
print("Positive Recall:", TP / (TP + FP))
print("Negative Recall:", TN / (TN + FN))

# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

Random image name: img_1001031.png


Processing:   0%|          | 0/80 [00:00<?, ?it/s]C:\Users\atashitk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\functional.py:1352: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "
Processing: 100%|██████████| 80/80 [00:47<00:00,  1.70it/s]


TP: 0.27 %
TN: 97.18 %
FP: 2.43 %
FN: 0.12 %
0.6947856675211114
0.9756086930689504
0.9745098625387654

Positive Recall: 0.10063570772280048
Negative Recall: 0.9987725705088794


In [35]:
input_dir = "./preparations/data/indata"
output_dir = "./dataset/preprocess/"
# Create the folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "indata"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "outdata"), exist_ok=True)

for image_file in tqdm(os.listdir(input_dir)):
    input_image_path = os.path.join(input_dir, image_file)
    output_image_path1 = os.path.join(output_dir, "indata", image_file)
    output_image_path2 = os.path.join(output_dir, "outdata", image_file)
    img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    output_image = img * (mask // 255)
    cv2.imwrite(output_image_path1, output_image)
    _, thresh = cv2.threshold(output_image, 41, 255, cv2.THRESH_BINARY)
    cv2.imwrite(output_image_path2, thresh)

100%|██████████| 1340/1340 [01:37<00:00, 13.76it/s]
